# Image API

The IIIF Image api defines standards for both requests for images and image data structures. This notebook will demonstrate both, using two examples: 
- The same [example document from Discovery](https://discovery.nationalarchives.gov.uk/details/r/C9060661)
- An live example from the Bodleian library [IIIF image server](https://iiif.bodleian.ox.ac.uk/iiif/image/1363b336-260d-4f22-a6cf-4e1320dbb689/full/256,/0/default.jpg). 

For the National Archives example, we will be building up the IIIF request from scratch to show how it might work - note that, at time of writing, TNA does not support IIIF, and the URIs built here will not work. The Bodleian example will show how a working example. 

The full documentation for the IIIF Image API can be found [here](https://iiif.io/api/image/3.0/).

In [ ]:
%pip install -q requests
%pip install -q IPython
%pip install -q json

import requests
from IPython.display import Image
import json

## Image requests

Requests for images come in the form of URIs, and can be independent requests (as in, a user puts the URL into a browser), but are more often used in the context of a IIIF viewer, or as part of a IIIF manifest. The URI contains all the information needed to request the image, with the standard defining what information is needed and how it should be formatted. This information is presented in 5 parts, appended onto a base url. The base URI should not actually request the image, instead it should point towards the resource - the additional information then tells the server exactly what view of the image is requested. For this example, we will be building the TNA example up, and adjusting the Bodleian example at each stage to show how the URI changes.

In [ ]:
TNA_base_url = "https://discovery.nationalarchives.gov.uk/details/r/C9060661/"

oxford_example_url = "https://iiif.bodleian.ox.ac.uk/iiif/image/1363b336-260d-4f22-a6cf-4e1320dbb689/full/256,/0/default.jpg"

Image(url=oxford_example_url)

### 1 - Region

The area of the base image being requested. The possible values are:
- `full` - the entire image
- `square` - using the shortest edge of the parent image to define a square region of the parent image - typically at the centre of the long edge.
- `x,y,w,h` - where x and y are the coordinates of the top left corner of the region, and w and h are the width and height of the requested region in pixels. Can be prefixed with `pct:` to use percentages instead of pixels.

In [ ]:
TNA_base_url += "square/"

oxford_square_url = "https://iiif.bodleian.ox.ac.uk/iiif/image/1363b336-260d-4f22-a6cf-4e1320dbb689/square/256,/0/default.jpg"

Image(url=oxford_square_url)

### 2 - Size 

The next part of the URI provides information on the size the image will be scaled to. There are a few possible values, with the most common being:

- `max`  - The image will be shown at the maximum size without exceeding the dimensions of the parent image. 
- `w,` - Defines the width of the image in pixels, with the height being calculated to maintain the aspect ratio of the parent image. Does not exceed the dimensions of the parent image.
- `,h` - As per width, but for height.
- `w,h` - Defining both the width and height will override the aspect ratio of the parent image, potentially distorting the image.

These can be prefixed with `^` to allow the viewer to scale the image larger than the parent. 

In [ ]:
TNA_base_url += "200,/"

oxford_100by100_url = "https://iiif.bodleian.ox.ac.uk/iiif/image/1363b336-260d-4f22-a6cf-4e1320dbb689/square/100,100/0/default.jpg"

Image(url=oxford_100by100_url)

### Rotation

The rotation of the image, in degrees. Adding a `!` before the rotation will mirror the image before rotating. Rotating by a value that is not a multiple of 90 will result in a rectangular image, padded with transparency if the format supports it (if the server can accommodate the request).

In [ ]:
TNA_base_url += "!270/"

oxford_rotated_url = "https://iiif.bodleian.ox.ac.uk/iiif/image/1363b336-260d-4f22-a6cf-4e1320dbb689/square/100,100/270/default.jpg"

Image(url=oxford_rotated_url)

### Quality

Quality refers to the colour of the image. 

- `color` - Full colour image
- `gray` - Greyscale image
- `bitonal` - Black and white image
- `default` - The server will provide the image in the format it is stored in. 

`Default` should always be supported by IIIF servers, and `color` (if supported) should return an image regardless of the colour depth of the original image.

In [ ]:
TNA_base_url += "color"

oxford_bitonal_url = "https://iiif.bodleian.ox.ac.uk/iiif/image/1363b336-260d-4f22-a6cf-4e1320dbb689/square/100,100/270/bitonal.jpg"

Image(url=oxford_bitonal_url)

### Format

Format is the final part of the URI, with common fileneame extensions being used; e.g. `jpg`, `png`, `gif`, `tif`, `webp`, `jp2`. The Bodleian example is also available in `webp` format.

In [ ]:
TNA_base_url += ".jpg"

print(TNA_base_url)

oxford_webp_url = "https://iiif.bodleian.ox.ac.uk/iiif/image/1363b336-260d-4f22-a6cf-4e1320dbb689/square/100,100/270/bitonal.webp"

Image(url=oxford_webp_url)

### Additional details about requests

Steps in the request are applied in order, and must be requested in the order shown here - so, for example, if you requested a square region, and a rotation, the square would be taken from the original image first, then the rotation would be applied to the square.

At each stage of the request, it is possible that the URI may specify something the server cannot provide - for example, the request may be for a `.jpg` format image, despite it only being available in `.tif`, or the size requested maybe larger than the original. In these cases, the server generally will respond with a 400 code error. 

## Information requests

The IIIF Image API specification also describes what information a server should provide when requested for information about an image. It is this information that allows the viewer (either a IIIF viewer, or the end user themselves) to form good requests for the image. The response is JSON, with fields specifying all the information needed to form a request, and additional details such as rights and metadata. Here, we are going to request the information for the image from Oxford, examine the response, and use it to build up a hypothetical document for the example from Discovery.

The v3 header added here comes from the [IIIF api documentation from the Bodleian](https://digital.bodleian.ox.ac.uk/developer/iiif/#tag/Image) - as they offer both IIIF v2 and v3 services, you need to specify which you want to use. This notebook assumes and works with v3, as it is the most recent version of the standard.

In [ ]:
oxford_eg = "https://iiif.bodleian.ox.ac.uk/iiif/image/1363b336-260d-4f22-a6cf-4e1320dbb689/info.json"

v3_header = "application/ld+json;profile=http://iiif.io/api/image/3/context.json"

r = requests.get(oxford_eg, headers={"Accept": v3_header})

print(json.dumps(r.json(), indent=2))

This full response can be broken down into a few key parts. 

### The default information/technical metadata

The standard specifies some information required for all images: 

- `@context` - The context of the response, which should be `http://iiif.io/api/image/3/context.json`
- `id` - The base URI of the resource - without information about the region, size, rotation, quality, or format.
- `type` - The type of the resource, which should be `ImageService3`
- `protocol` - The version of the IIIF Image API the server is using, which should be `http://iiif.io/api/image`
- `profile` - The level (from 1-3) of data the server can provide.
- `width` - The width of the full image in pixels.
- `height` - The height of the full image in pixels.

The response from Oxford includes two additional, optional fields:

- `maxWidth` and `maxHeight` - The maximum width and height of the image in pixels.

Using the Discovery example, with just the required fields and some example values could look like the below. 

In [ ]:
discovery_example = {
    "@context": "http://iiif.io/api/image/3/context.json",
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661", # <-- the base location of the image - without a trailing slash or other information
    "type": "ImageService3", 
    "protocol": "http://iiif.io/api/image", 
    "profile": "level 1", 
    "width": 1000,
    "height": 1000, 
}

### Sizes

Sizes are an optional field, included in the response from Oxford. This field lists the sizes the server can provide the image in, and is provided as a list of objects, each with a `width` and `height` field - specified in pixels. The information for the Oxford image indicates that it can be provided in 5 sizes, each 50% smaller than the previous. 

This snippet would indicate that the Discovery example was available only additionally available at 50% the original size.

In [ ]:
sizes = [
    {
        "width": 500,
        "height": 500
    }
]

### Tiles

The zoom functionality of IIIF viewers is provided by the server providing the image in a series of tiles, which are then stitched together by the viewer. The `tiles` field in the response provides information about these tiles, including the size, number of levels, and width and height in pixels. We are going to ignore this field for the Discovery example. 

### Extra functionality

There are three fields that can be added to indicate extra functionality options the server can provide. The data these fields contain provides information that specifies options that can be used in the request URI detailed above.

- `extraQualities` - A list of additional qualities (colour formats) the server can provide the image in. In the example from Oxford, 'color' 'grey' and 'bitonal' are provided.
- `extraFormats` - A list of additional formats the server can provide the image in, such as `tif`. The Oxford example provides just 'webp'. 
- `extraFeatures` - A list of additional features the server can provide, such as `rotationArbitrary` or `mirroring`. The list of features can be found on [the IIIF website here](https://iiif.io/api/image/3.0/#57-extra-functionality), where details of the options provided by Oxford can be decoded.

For the Discovery-based example, we will include a simple option for each of these fields.

In [ ]:
extra_qualities = [
    "bitonal" 
]
extra_formats = [
    "webp" 
]
extra_features = [
    "rotationArbitrary" # indicating the server can rotate the image to any angle
]

### Other information

There are other fields that can also be included in the response that are not in the Oxford example; 

- `rights` - using a URL from, for example, Creative Commons, to specify the rights of the image.
- Options to link to other resources:
    -  `partOf` (something refering to this image, such as a [canvas](./presentation_new.ipynb)), 
    - `seeAlso` (a link to a resource with more information about the image),
    - `service` (an external service related to the resource, providing functionality, such as authentication). 

In the [presentation notebook](./presentation_new.ipynb), we used the same example from Discovery to show how to build a IIIF manifest, so here we need to provide a `partOf` link to the appropriate canvas.

In [ ]:
part_of = [
    {
        "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/manifest.json", #<- part of can point to a canvas or a manifest, here we are pointing to a manifest,
                                                                                            # as the resource is both pages
        "type": "Manifest"
    }
]

### The full response

The fully constructed response for the Discovery, then, would look like the below.

In [ ]:
discovery_example["sizes"] = sizes
discovery_example["extraQualities"] = extra_qualities
discovery_example["extraFormats"] = extra_formats
discovery_example["extraFeatures"] = extra_features
discovery_example["partOf"] = part_of

print(json.dumps(discovery_example, indent=2))